In [ ]:
import json
import logging
import hmac
import hashlib
import urllib.parse
import urllib.request
import os
import datetime

# Initialize logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# AWS Region and Service Endpoint
REGION = "us-east-1"
SERVICE = "cloudshell"
ENDPOINT = f"https://cloudshell.{REGION}.amazonaws.com"

# Sign AWS requests (AWS Signature Version 4)
def sign(key, msg):
    return hmac.new(key, msg.encode('utf-8'), hashlib.sha256).digest()

def get_signature_key(secret_key, date_stamp, region_name, service_name):
    k_date = sign(('AWS4' + secret_key).encode('utf-8'), date_stamp)
    k_region = sign(k_date, region_name)
    k_service = sign(k_region, service_name)
    k_signing = sign(k_service, 'aws4_request')
    return k_signing

def create_signed_headers(action, payload):
    access_key = os.environ.get("AWS_ACCESS_KEY_ID")
    secret_key = os.environ.get("AWS_SECRET_ACCESS_KEY")
    session_token = os.environ.get("AWS_SESSION_TOKEN")

    if not access_key or not secret_key:
        raise ValueError("Missing AWS credentials in Lambda environment variables.")

    # Date and time formatting
    t = datetime.datetime.utcnow()
    amz_date = t.strftime('%Y%m%dT%H%M%SZ')
    date_stamp = t.strftime('%Y%m%d')

    # Create canonical headers
    canonical_uri = "/"
    canonical_querystring = ""
    canonical_headers = (
        f"content-type:application/x-amz-json-1.0\n"
        f"host:cloudshell.{REGION}.amazonaws.com\n"
        f"x-amz-date:{amz_date}\n"
        f"x-amz-target:AWSCloShellService.{action}\n"
    )
    signed_headers = "content-type;host;x-amz-date;x-amz-target"

    # Hash payload
    payload_hash = hashlib.sha256(payload.encode("utf-8")).hexdigest()

    # Create canonical request
    canonical_request = (
        f"POST\n{canonical_uri}\n{canonical_querystring}\n{canonical_headers}\n"
        f"{signed_headers}\n{payload_hash}"
    )

    # Create string to sign
    algorithm = "AWS4-HMAC-SHA256"
    credential_scope = f"{date_stamp}/{REGION}/{SERVICE}/aws4_request"
    string_to_sign = (
        f"{algorithm}\n{amz_date}\n{credential_scope}\n"
        f"{hashlib.sha256(canonical_request.encode('utf-8')).hexdigest()}"
    )

    # Sign the string
    signing_key = get_signature_key(secret_key, date_stamp, REGION, SERVICE)
    signature = hmac.new(signing_key, string_to_sign.encode("utf-8"), hashlib.sha256).hexdigest()

    # Create authorization header
    authorization_header = (
        f"{algorithm} Credential={access_key}/{credential_scope}, "
        f"SignedHeaders={signed_headers}, Signature={signature}"
    )

    # Construct headers
    headers = {
        "Content-Type": "application/x-amz-json-1.0",
        "X-Amz-Date": amz_date,
        "X-Amz-Target": f"AWSCloShellService.{action}",
        "Authorization": authorization_header,
    }

    if session_token:
        headers["X-Amz-Security-Token"] = session_token

    return headers

def make_cloudshell_request(action, payload):
    try:
        # Generate signed headers
        headers = create_signed_headers(action, payload)

        # Make the request
        request = urllib.request.Request(
            ENDPOINT,
            data=payload.encode("utf-8"),
            headers=headers,
            method="POST",
        )
        logger.info(f"Request Headers: {headers}")
        logger.info(f"Request Payload: {payload}")

        with urllib.request.urlopen(request) as response:
            response_data = response.read().decode("utf-8")
            logger.info(f"Response Data: {response_data}")
            return {"statusCode": response.status, "body": json.loads(response_data)}

    except Exception as e:
        logger.error(f"Error making CloudShell request: {str(e)}")
        return {"statusCode": 500, "body": {"error": str(e)}}

# Lambda handler
def lambda_handler(event, context):
    logger.info(f"Received event: {json.dumps(event)}")

    # Instead of processing the event or calling CloudShell,
    # simply return a success message.
    return {
        "statusCode": 200,
        "body": {
            "message": "Success!"
        }
    }
